In [333]:
%load_ext autoreload
%autoreload 2
import sys,os; sys.path.append(os.environ['BMESAHMETDIR']); import bmes

import sys,os; sys.path.append(os.environ['BMESAHMETDIR']+'/qsb'); from bmes_targetscandb_mir2target import *

import GEOparse
import pandas as pd
import random
from sklearn import svm
import numpy as np
from hwmaml_breastcancer_trainandtest import hwmaml_breastcancer_trainandtest
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
import statistics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download and parse the dataset

In [4]:
filegeo = "GSE7390"
arr = os.listdir(bmes.tempdir())

#Check if file exists in tempdir with file name
flag = 1
for file in arr:
    result = file.find(filegeo)
    if (result == 0):
        flag = 0

#If file does not exist, download file
if (flag == 1):
    gse = GEOparse.get_GEO(geo=filegeo, destdir=bmes.tempdir());
else:
    gse = GEOparse.get_GEO(filepath=bmes.tempdir() + '/' + filegeo + '_family.soft.gz')

08-May-2022 11:48:43 DEBUG utils - Directory C:/Users/user/AppData/Local/Temp/bmes already exists. Skipping.
08-May-2022 11:48:43 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE7nnn/GSE7390/soft/GSE7390_family.soft.gz to C:/Users/user/AppData/Local/Temp/bmes\GSE7390_family.soft.gz
100%|██████████| 42.7M/42.7M [00:01<00:00, 27.4MB/s]
08-May-2022 11:48:46 DEBUG downloader - Size validation passed
08-May-2022 11:48:46 DEBUG downloader - Moving C:\Users\user\AppData\Local\Temp\tmpe8jkhfb_ to C:\Users\user\AppData\Local\Temp\bmes\GSE7390_family.soft.gz
08-May-2022 11:48:46 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE7nnn/GSE7390/soft/GSE7390_family.soft.gz
08-May-2022 11:48:46 INFO GEOparse - Parsing C:/Users/user/AppData/Local/Temp/bmes\GSE7390_family.soft.gz: 
08-May-2022 11:48:46 DEBUG GEOparse - DATABASE: GeoMiame
08-May-2022 11:48:46 DEBUG GEOparse - SERIES: GSE7390
08-May-2022 11:48:46 DEBUG GEOparse - PLATFORM: GPL96
08-Ma

08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178014
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178015
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178016
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178017
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178018
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178019
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178020
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178021
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178022
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178023
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178024
08-May-2022 11:48:57 DEBUG GEOparse - SAMPLE: GSM178025
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GSM178026
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GSM178027
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GSM178028
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GSM178029
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GSM178030
08-May-2022 11:48:58 DEBUG GEOparse - SAMPLE: GS

In [10]:
gsedata = None
names = []

#Store data into table
for gsmid in gse.gsms.keys():
    
    #Apeend names to table
    names.append(gsmid)
    gsmdata = gse.gsms[gsmid].table.rename(columns={'VALUE':gsmid});
    
    if gsedata is None: gsedata=gsmdata;
    else:
        assert(gsedata['ID_REF'].equals(gsmdata['ID_REF'])) #just make sure that the same probes are listed in the same order.
        gsedata = pd.concat([gsedata,gsmdata[gsmid]],axis=1);
        
gsedata.head()

,ID_REF,GSM177885,GSM177886,GSM177887,GSM177888,GSM177889,GSM177890,GSM177891,GSM177892,GSM177893,...,GSM178073,GSM178074,GSM178075,GSM178076,GSM178077,GSM178078,GSM178079,GSM178080,GSM178081,GSM178082
0,1007_s_at,10.780787,11.335393,11.028074,11.847736,12.359239,11.086998,11.681446,11.232548,11.588695,...,11.626131,12.095386,10.818799,10.649233,10.619263,10.785023,11.390198,10.735085,10.653104,11.310992
1,1053_at,8.674201,9.354759,9.053889,9.139895,9.196708,8.808035,9.395361,8.793770,9.073383,...,9.252529,8.594601,9.372028,8.846761,8.639552,10.012078,9.830344,8.894038,8.854766,9.674872
2,117_at,7.738589,7.763657,6.327600,7.032598,7.873428,6.140035,7.401411,7.338878,6.597228,...,7.534337,7.973816,7.052388,7.072636,6.240142,6.855161,7.150656,7.262618,6.636791,7.276915
3,121_at,9.285511,9.025809,9.234409,9.656416,9.174188,9.283764,8.936069,9.363308,9.949485,...,8.949350,9.265321,9.373517,8.685993,9.227666,8.987488,8.766830,9.064142,9.283177,9.646810
4,1255_g_at,6.610484,4.809869,4.621973,5.589404,5.325173,2.995857,5.489110,5.604046,2.084769,...,3.653802,5.337356,5.524546,5.887738,5.316872,5.833270,3.977743,3.781916,3.724769,4.854293


# Get GSM Cancer relapse status

In [152]:
#Store relapse status
erfs = []

#For each gsm find the e.rfs value within characteristics_ch1
for gsm_name, gsm in gse.gsms.items():
    
    #Pull characteristics channel from metadata
    erfs_raw = gsm.metadata['characteristics_ch1'][14]
    
    #Split text from value
    split_erfs = erfs_raw.split(" ")
    
    #Convert to integer and append to erfs
    erfs.append(int(split_erfs[1]))

#First 5 relapse status
print(erfs[0:5])

[1, 1, 1, 1, 1]


# Filter 76 Genes

In [153]:
#Read stored excel file containing 76 genes to keep as features
Gene_76 = pd.read_excel('76 genes from the prognostic signature.xlsx', index_col=None, header=0)

#Convert dataframe to list of genes
Genes = list(Gene_76.columns.values)

#Index of 76 genes
i1 = list(Gene_76.set_index(Genes).index)

#Filter out all other genes
Filter_gsedata = gsedata[gsedata['ID_REF'].isin(i1)]

#First 5 genes kept with sample for column and gene as row
Filter_gsedata.head()

,ID_REF,GSM177885,GSM177886,GSM177887,GSM177888,GSM177889,GSM177890,GSM177891,GSM177892,GSM177893,...,GSM178073,GSM178074,GSM178075,GSM178076,GSM178077,GSM178078,GSM178079,GSM178080,GSM178081,GSM178082
253,200726_at,10.926361,12.242090,11.661716,12.174021,11.484011,11.548628,11.558369,12.277288,12.266570,...,11.203620,11.537107,11.931666,12.103354,11.957850,12.018292,11.711415,11.939616,11.848449,11.425778
492,200965_s_at,8.962608,9.531718,10.238680,9.819279,11.489233,11.188773,10.110021,11.029372,10.934970,...,11.898593,9.692620,9.276641,10.156689,10.661887,8.323876,10.428482,9.615587,10.528911,9.901486
595,201068_s_at,11.630078,12.626106,12.572919,12.109888,11.779285,11.887296,11.719263,11.727464,11.636529,...,12.207723,11.900644,12.404526,12.221951,12.150965,11.955274,12.420877,11.962812,11.318453,12.167550
618,201091_s_at,10.964107,11.594716,9.166088,9.086937,8.887616,9.853284,9.803192,10.376447,10.545703,...,10.586759,9.951362,10.068319,9.943352,10.355328,10.740020,11.145993,10.463171,8.609631,9.011730
815,201288_at,11.518305,12.317659,11.698658,13.132617,10.429663,12.979525,12.728942,13.029538,10.274986,...,12.282550,12.325630,12.831449,12.561559,12.302234,11.150428,11.084685,11.514539,13.719035,12.013692


# Normalize Data

In [239]:
#Mean of rows
gseMean = Filter_gsedata.mean(axis=1)

#Remove ID_ref to be left with only expression data
removedID = Filter_gsedata.loc[:, Filter_gsedata.columns != 'ID_REF']

#substract each value from row mean
normalized_gsedata= removedID.sub(gseMean, axis=0)

#Reinsert ID_REF
normalized_gsedata.insert(loc=0, column='ID_REF', value=Filter_gsedata['ID_REF'])

# Split into Training and Testing Datasets

In [240]:
#Number of samples
numSamples = len(normalized_gsedata.columns)

#Store Samples for training set
TrainingCol = []

#Store Samples for testing set
TestingCol = []

#Column names list
ColNames = list(normalized_gsedata.columns)

#Storing label values for training
TrainY = []

#Storing label values for testing
TestY = []

#Iterate through samples
#Randomly select integer 1 to 10
for i in range(1, numSamples):
    randnum = random.randint(1, 10)
    
    #If value is 10 put sample in testing set
    if (randnum == 10):
        TestingCol.append(ColNames[i])
        TestY.append(erfs[i-1])
        
    #If value is less than 10 put sample in training set
    elif (randnum < 10):
        TrainingCol.append(ColNames[i])
        TrainY.append(erfs[i-1])
        
#Dataframe for training set
TrainingSet = pd.DataFrame(columns = i1)

#Dataframe for testing set
TestingSet = pd.DataFrame(columns = i1)

#Iterate through genes and put sampels into correct set
for i in range(len(i1)):
    
    #Current Gene
    Gene = i1[i]
    
    #Find row containg gene
    GeneRow = normalized_gsedata.loc[normalized_gsedata['ID_REF'] == Gene]
    
    #Keep only samples for training set
    TrainingFilter = GeneRow.filter(TrainingCol)
    
    #Keep only samples for testing set
    TestingFilter = GeneRow.filter(TestingCol)
    
    #Convert dataframe to matrix
    TrainingMat = TrainingFilter.to_numpy()
    
    #Convert dataframe to matrix
    TestingMat = TestingFilter.to_numpy()
    
    #Add training samples to dataframe
    TrainingSet[Gene] = TrainingMat[0]
    
    #Add testing samples to dataframe
    TestingSet[Gene] = TestingMat[0]

# Train SVM Model

In [241]:
#Convert dataframe to matrix
TrainX = TrainingSet.to_numpy()

#Create svm variable
clf = svm.SVC()

#Fit data to svm
clf.fit(TrainX, TrainY)

SVC()

# Get SVM Predictions

In [275]:
#Number of samples in testing set
numTest = len(TestY)

#Store correct number of predictions
correct_count = 0

#For each sample predict label
for i in range(numTest):
    
    #Select row from testing set
    row = TestingSet.iloc[[i]]
    
    #Convert to matrix
    TestX = row.to_numpy()
    
    #Predict label
    result = clf.predict(TestX)
    
    #If prediction is correct add to correct count
    if (result[0] == TestY[i]):
        correct_count += 1

#Calculate accuracy
accuracy = correct_count / numTest

#Report accuracy
print(accuracy)

0.5652173913043478


# Predict Class with hwmaml_breastcancer_trainandtest

In [296]:
#Convert testing set to matrix
TestX = TestingSet.to_numpy()

#run hwmaml_breastcancer_trainandtest to get number of errors
numerror = hwmaml_breastcancer_trainandtest(TrainX, TrainY, TestX, TestY)

#Calculate accuracy based on errors
accuracy = 1 - numerror/numTest

#Report accuracy
print(accuracy)

0.5652173913043479


# Perform 10-fold cross-validation

In [334]:
#Remove ID_Ref from dataframe
removedID = normalized_gsedata.loc[:, normalized_gsedata.columns != 'ID_REF']

#Transpose dataframe
gsedata_normalized_transposed = removedID.T

#Convert Dataframe to matrix
Normalized_Mat = gsedata_normalized_transposed.to_numpy()

#Number of cross fold iterations
cf = 10

#Store accuracies
accuracies = []

#Run 10 crossfolds
for i in range(cf):
    
    #Use train_test_split to split into 90% training 10% testing
    X_train, X_test, y_train, y_test = train_test_split( Normalized_Mat, erfs, test_size=0.1, random_state=0)
    
    #Create svm variable
    clf = svm.SVC()
    
    #Use sequential feature selector to pick half of the features that are most useful in predicting
    sfs = SequentialFeatureSelector(clf,
        n_features_to_select=None, 
        direction='forward')
    
    #Fit sfs
    sfs.fit(X_train, y_train)
    
    #Use sfs to transform training and testing sets
    X_train_sfs = sfs.transform(X_train)
    X_test_sfs = sfs.transform(X_test)
    
    #Run model and Calculate number of errors
    numerror = hwmaml_breastcancer_trainandtest(X_train_sfs, y_train, X_test_sfs, y_test)
    
    #Calculate accuracy for crossfold
    accuracy = 1 - numerror/numTest
    
    #Add accuracy to list of accuracies
    accuracies.append(accuracy)

#Calculate total mean between cross
TotalAccuracy = statistics.mean(accuracies)

#Report total accuracy
print(TotalAccuracy)

0.6956521739130435
